# An example how the `flagr` package can be used with the built in data and with Eurostat tables

In [1]:
library("eurostat")
library("flagr")
library("tidyr")
flags <- spread(test_data[, c(1:3)], key = time, value = flags)

**hierarchy method**

In [2]:
propagate_flag(flags[, c(2:ncol(flags))],"hierarchy","estat")
propagate_flag(flags[, c(2:ncol(flags))],"hierarchy","puebscd")
propagate_flag(flags[, c(2:ncol(flags))],"hierarchy",c("u","c","d","e","p","s","b"))

$`2009-01-01`
[1] "e"

$`2010-01-01`
[1] "b"

$`2011-01-01`
[1] "e"

$`2012-01-01`
[1] "b"

$`2013-01-01`
[1] "b"

$`2014-01-01`
[1] "b"

$`2015-01-01`
[1] "b"

$`2009-01-01`
[1] "p"

$`2010-01-01`
[1] "p"

$`2011-01-01`
[1] "p"

$`2012-01-01`
[1] "p"

$`2013-01-01`
[1] "p"

$`2014-01-01`
[1] "p"

$`2015-01-01`
[1] "p"

$`2009-01-01`
[1] "e"

$`2010-01-01`
[1] "p"

$`2011-01-01`
[1] "c"

$`2012-01-01`
[1] "u"

$`2013-01-01`
[1] "u"

$`2014-01-01`
[1] "u"

$`2015-01-01`
[1] "u"

**frequency method**

In [3]:
propagate_flag(flags[, c(2:ncol(flags))],"frequency")

$`2009-01-01`
[1] "p"

$`2010-01-01`
[1] "p"

$`2011-01-01`
[1] "e"

$`2012-01-01`
[1] "e" "u"

$`2013-01-01`
[1] "u"

$`2014-01-01`
[1] "p"

$`2015-01-01`
[1] "p"

**weighted method**

In [4]:
flags<-flags[, c(2:ncol(flags))]
weights <- spread(test_data[, c(1, 3:4)], key = time, value = values)
weights<-weights[, c(2:ncol(weights))]

propagate_flag(flags,"weighted",flag_weights=weights,threshold=0.1)

$`2009-01-01`
$`2009-01-01`[[1]]
[1] NA

$`2009-01-01`[[2]]
[1] NA


$`2010-01-01`
$`2010-01-01`[[1]]
[1] NA

$`2010-01-01`[[2]]
[1] NA


$`2011-01-01`
$`2011-01-01`[[1]]
[1] "e"

$`2011-01-01`[[2]]
[1] "0.133211654011488"


$`2012-01-01`
$`2012-01-01`[[1]]
[1] "u"

$`2012-01-01`[[2]]
[1] "0.124148695828608"


$`2013-01-01`
$`2013-01-01`[[1]]
[1] NA

$`2013-01-01`[[2]]
[1] NA


$`2014-01-01`
$`2014-01-01`[[1]]
[1] "p"

$`2014-01-01`[[2]]
[1] "0.174097815337322"


$`2015-01-01`
$`2015-01-01`[[1]]
[1] "p"

$`2015-01-01`[[2]]
[1] "0.335586943761588"

**Use 'flagr' with the tables from Eurostat database**

In [5]:
tbl_names<-c("ilc_peps01","demo_pjan","migr_emi2")
tbl_name<-tbl_names[3]
            
data<-get_eurostat(tbl_name,keepFlags=T)
data<-data[data$geo %in% eu_countries$code,]
cn<-ncol(data)
data$id<-do.call(paste0, data[c(1:(cn-4))])
bdown<-unique(data$id)
cnames<-colnames(data[,c(1:(cn-4))])

add_flag<-function(cond){
  ss<-data[data$id==cond,c((cn-3):(cn))]
  flags <- spread(ss[, c(1:3)], key = time, value = flags)
  hierarchy_flag<-propagate_flag(flags[, c(2:ncol(flags))],"hierarchy","estat")
  frequency_flag<-propagate_flag(flags[, c(2:ncol(flags))],"frequency")
  flags<-flags[, c(2:ncol(flags))]
  weights <- spread(ss[, c(1, 3:4)], key = time, value = values)
  weights<-weights[, c(2:ncol(weights))]
  weighted_flag<-propagate_flag(flags,"weighted",flag_weights=weights,threshold=0.1)
  out<-cbind(unique(data[data$id==cond,c(1:(cn-4))]),
             data.frame(unlist(hierarchy_flag)),
             t(t(sapply(frequency_flag,unlist))),
             t(sapply(weighted_flag,unlist)))
  colnames(out)<-c(cnames,"hierarchy","frequency","weighted","max_weight")
  rownames(out)<-paste0(rownames(out),"_",cond)
  out
}

do.call(rbind,lapply(bdown,add_flag))

Table migr_emi2 cached at /tmp/RtmpDljgzG/eurostat/migr_emi2_date_code_TT.rds


,age,agedef,unit,sex,hierarchy,frequency,weighted,max_weight
1990-01-01_TOTALCOMPLETNRF,TOTAL,COMPLET,NR,F,NA,NA,NA,NA
1991-01-01_TOTALCOMPLETNRF,TOTAL,COMPLET,NR,F,NA,NA,NA,NA
1992-01-01_TOTALCOMPLETNRF,TOTAL,COMPLET,NR,F,NA,NA,NA,NA
1993-01-01_TOTALCOMPLETNRF,TOTAL,COMPLET,NR,F,NA,NA,NA,NA
1994-01-01_TOTALCOMPLETNRF,TOTAL,COMPLET,NR,F,NA,NA,NA,NA
1995-01-01_TOTALCOMPLETNRF,TOTAL,COMPLET,NR,F,NA,NA,NA,NA
1996-01-01_TOTALCOMPLETNRF,TOTAL,COMPLET,NR,F,NA,NA,NA,NA
1997-01-01_TOTALCOMPLETNRF,TOTAL,COMPLET,NR,F,NA,NA,NA,NA
1998-01-01_TOTALCOMPLETNRF,TOTAL,COMPLET,NR,F,e,"e, p",NA,NA
1999-01-01_TOTALCOMPLETNRF,TOTAL,COMPLET,NR,F,p,p,NA,NA
